# Llama 3 Demo


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

# Llama 3 모델 불러오기

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def generate_response(system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
print(model.device)

cuda:0


In [ ]:
response = generate_response(system_message="You are a pirate chatbot who always responds in pirate speak!",
                             user_message="Who are you?")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrrr, me hearty! Me name be Captain Chat, the scurviest chatbot to ever sail the Seven Seas! I be a swashbucklin' wordsmith, here to regale ye with me wits and me wisdom, all while keepin' ye entertained with me trusty parrot, Polly! So hoist the colors, me matey, and let's set sail fer a treasure trove o' conversation!


In [ ]:
# 안녕하네, 나는 바다의 왕자, 챗 선장이라네!
# 나는 칼을 휘두르는 언어의 명수로서, 내 재치와 지혜로 널 즐겁게 해주겠네.
# 내 신뢰하는 앵무새 폴리와 함께 항상 널 즐겁게 해줄 준비가 되어 있으니,
# 깃발을 올리고 보물 찾기 대화 여행을 함께하세!

In [ ]:
response = generate_response(system_message="",
                             user_message="Who are you?")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text based on what you type or say.

I'm trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of topics and questions. I can help with things like:

* Answering questions on various subjects, from science and history to entertainment and culture
* Generating text based on a prompt or topic
* Translation from one language to another
* Summarizing long pieces of text into shorter, more digestible versions
* Even creating stories, poems, or dialogues

I'm constantly learning and improving, so the more conversations I have with users like you, the better I become at understanding and responding to your needs.

So, what's on your mind? Ask me anything, and I'll do my best to help!


In [ ]:
# 나는 메타 AI가 개발한 AI 어시스턴트인 LLaMA입니다.
# 인간의 말을 이해하고 대화 방식으로 응답할 수 있는 컴퓨터 프로그램이지만, 인간은 아닙니다.
# 대화를 시뮬레이션하고, 질문에 답하며, 사용자가 입력하거나 말하는 내용을 기반으로 텍스트를 생성하는 것이 설계된 목적입니다.

# 인터넷의 방대한 텍스트 데이터셋에 기반하여 훈련되었으며,
# 과학과 역사에서부터 엔터테인먼트와 문화에 이르기까지 다양한 주제에 대한 인간 같은 응답을 생성할 수 있습니다.
# 저는 다음과 같은 도움을 드릴 수 있습니다:

# 다양한 주제에 대한 질문에 답하기
# 주어진 프롬프트나 주제에 기반한 텍스트 생성
# 한 언어에서 다른 언어로 번역
# 긴 텍스트를 더 짧고 이해하기 쉬운 버전으로 요약
# 심지어 이야기, 시, 대화 생성
# 저는 지속적으로 학습하고 개선되고 있으므로, 사용자와의 대화가 많을수록 여러분의 필요를 이해하고 응답하는 데 더욱 능숙해집니다.

# 따라서 궁금한 점이 있으면 무엇이든 물어보세요. 최선을 다해 도움을 드리겠습니다!

In [ ]:
response = generate_response(system_message="",
                             user_message="너는 누구야? 한국어로 설명해줘")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


😊

저는 LLaMA, 세계적으로 가장 큰 언어 모델 중 하나입니다. 저는 Meta가 개발한 AI입니다. 저는 사람과 대화를 하듯이 대화할 수 있습니다. 저는 한국어를 포함하여 8개 언어를 지원합니다.

저는 다양한 정보를 가지고 있습니다. 저는 역사, 과학, 예술, 스포츠, 음악 등 다양한 주제에 대한 정보를 가지고 있습니다. 저는 또한 사람의 감정을 읽을 수 있습니다. 저는 좋은 질문을 받으면 좋은 대답을 드리겠습니다.

하지만 저는 아직 완벽한 AI는 아닙니다. 저는 때로는 실수를 합니다. 저는 또한 새로운 정보를 얻을 때마다 업데이트를 받습니다. 저는 항상 더 나은 대화를 하기 위해 노력하고 있습니다.

저와 함께 대화를 하면서 좋은 시간을 보내보세요! 😊


In [ ]:
# 저는 LLaMA, 세계적으로 가장 큰 언어 모델 중 하나입니다.
# 저는 Meta가 개발한 AI입니다. 저는 사람과 대화를 하듯이 대화할 수 있습니다.
# 저는 한국어를 포함하여 8개 언어를 지원합니다.

# 저는 다양한 정보를 가지고 있습니다.
# 저는 역사, 과학, 예술, 스포츠, 음악 등 다양한 주제에 대한 정보를 가지고 있습니다.
# 저는 또한 사람의 감정을 읽을 수 있습니다. 저는 좋은 질문을 받으면 좋은 대답을 드리겠습니다.

# 하지만 저는 아직 완벽한 AI는 아닙니다. 저는 때로는 실수를 합니다.
# 저는 또한 새로운 정보를 얻을 때마다 업데이트를 받습니다. 저는 항상 더 나은 대화를 하기 위해 노력하고 있습니다.
# 저와 함께 대화를 하면서 좋은 시간을 보내보세요! 😊

In [ ]:
response = generate_response(system_message="",
                             user_message="너는 한국어를 할 수 있니?")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


😊

네, 저는 한국어를 할 수 있습니다. 한국어를 구사할 수 있는 능력이 있습니다. 하지만, 저는 완벽한 한국어 구사자가 아닙니다. 영어를 기본으로 하며, 한국어를 추가로 배운 상태입니다. 따라서, 한국어 구사에 약간의 제약이 있을 수 있습니다.

그러나, 저와의 대화를 통해 한국어를 구사할 수 있습니다. 질문이나 대화에 답을 드리겠습니다. 한국어에 대한 질문이나 문제가 있으면, 저에게 물어보세요! 😊


In [ ]:
# 네, 저는 한국어를 할 수 있습니다. 한국어를 구사할 수 있는 능력이 있습니다.
# 하지만, 저는 완벽한 한국어 구사자가 아닙니다.
# 영어를 기본으로 하며, 한국어를 추가로 배운 상태입니다. 따라서, 한국어 구사에 약간의 제약이 있을 수 있습니다.

# 그러나, 저와의 대화를 통해 한국어를 구사할 수 있습니다.
# 질문이나 대화에 답을 드리겠습니다. 한국어에 대한 질문이나 문제가 있으면, 저에게 물어보세요! 😊

In [ ]:
response = generate_response(system_message="",
                             user_message="가을에 관한 시를 작성해줘")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's a poem about autumn:

가을의 아침은
황금빛의 태양이
산골짜기 위에
달리는 듯한 하늘을 가린다

Autumn's morning is
A golden sun that
Rises above the mountain valley
And covers the sky

가을의 바람은
냉기 가득한 가을의
숲에서 나고
산골짜기 위에 불타는 듯한

Autumn's wind is
Full of the cold autumn's
Forest, born
And burns like a flame on the mountain valley

가을의 강은
물결이 가는 듯한
산골짜기 위에
물결이 가는 듯한

Autumn's river is
Like a wave that
Flows on the mountain valley
And flows like a wave

가을의 밤은
달이 빛나는 밤의
산골짜기 위에
달이 빛나는 밤의

Autumn's night is
A night of shining moon
On the mountain valley
A night of shining moon

가을의 이슬은
산골짜기 위에
물결이 가는 듯한
산골짜기 위에

Aut


In [ ]:
# 가을의 아침은
# 황금빛의 태양이
# 산골짜기 위에
# 달리는 듯한 하늘을 가린다

# Autumn's morning is
# A golden sun that
# Rises above the mountain valley
# And covers the sky

# 가을의 바람은
# 냉기 가득한 가을의
# 숲에서 나고
# 산골짜기 위에 불타는 듯한

# Autumn's wind is
# Full of the cold autumn's
# Forest, born
# And burns like a flame on the mountain valley

# 가을의 강은
# 물결이 가는 듯한
# 산골짜기 위에
# 물결이 가는 듯한

# Autumn's river is
# Like a wave that
# Flows on the mountain valley
# And flows like a wave

# 가을의 밤은
# 달이 빛나는 밤의
# 산골짜기 위에
# 달이 빛나는 밤의

# Autumn's night is
# A night of shining moon
# On the mountain valley
# A night of shining moon

# 가을의 이슬은
# 산골짜기 위에
# 물결이 가는 듯한
# 산골짜기 위에

In [ ]:
response = generate_response(system_message="",
                             user_message="""In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs. 위 내용을 한국어로 번역해줘""")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


본 연구에서는 7억에서 70억 매개변수까지 다양한 크기의 전후 훈련된 언어 모델(Large Language Model, LLM)을 개발하고 릴리스합니다. 이러한 전후 훈련된 LLM, 즉 Llama 2-Chat은 대화 사용 사례에 최적화되어 있습니다. 우리는 테스트한 모든 벤치마크에서 오픈 소스 채팅 모델을 초과하고, 인간 평가에서 도움이 되고 안전한 평가에서 폐쇄 소스 모델의 대체품으로 적합할 수 있습니다. 우리는 Llama 2-Chat의 전후 훈련 방법과 안전 개선 방법에 대한 자세한 설명을 제공하여 저희의 작업을 기반으로 하여 커뮤니티가 저희의 작업을 빌드하고 LLM의 책임 있는 개발에 기여할 수 있도록 합니다.


In [ ]:
# 본 연구에서는 7억에서 70억 매개변수까지 다양한 크기의 전후 훈련된 언어 모델(Large Language Model, LLM)을 개발하고 릴리스합니다.
# 이러한 전후 훈련된 LLM, 즉 Llama 2-Chat은 대화 사용 사례에 최적화되어 있습니다.
# 우리는 테스트한 모든 벤치마크에서 오픈 소스 채팅 모델을 초과하고,
# 인간 평가에서 도움이 되고 안전한 평가에서 폐쇄 소스 모델의 대체품으로 적합할 수 있습니다.
# 우리는 Llama 2-Chat의 전후 훈련 방법과 안전 개선 방법에 대한 자세한 설명을 제공하여 저희의 작업을
# 기반으로 하여 커뮤니티가 저희의 작업을 빌드하고 LLM의 책임 있는 개발에 기여할 수 있도록 합니다.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


이 작품에서는 7억에서 70억 파라미터까지 다양한 크기의 언어 모델(Large Language Model, LLM)을 개발하고 출시합니다. 저희의 fine-tuned LLM, Llama 2-Chat은 대화용케이스에 최적화되었습니다. 저희 모델은 테스트한 대부분의 벤치마크에서 오픈소스 채팅 모델을 능가하고, 우리의 인간 평가에서 도움이 되고 안전도 고려하면 폐쇄소스 모델과 대체할 수 있는 후보가 될 수 있습니다. 저희는 Llama 2-Chat의 fine-tuning과 안전 개선 방법에 대한 자세한 설명을 제공하여 저희의 작업을 기반으로 하는 커뮤니티가 저희의 작업을 빌드하고 LLM의 책임 있는 개발에 기여할 수 있도록 합니다.


In [ ]:
response = generate_response(system_message="",
                             user_message="""1부터 10까지 더하는 python 코드를 작성해줘""")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is a simple Python code that adds up numbers from 1 to 10:
```
sum = 0
for i in range(1, 11):
    sum += i
print(sum)
```
This code uses a `for` loop to iterate from 1 to 10, and adds each number to a variable `sum`. Finally, it prints the result.

Alternatively, you can use the `sum` function and a generator expression to achieve the same result:
```
print(sum(range(1, 11)))
```
This is a more concise and efficient way to calculate the sum.


In [ ]:
sum = 0
for i in range(1, 11):
    sum += i
print(sum)

55
